In [1]:

import cv2 
import numpy as np
from matplotlib import pyplot as plt
import operator
import pytesseract

In [2]:

img = cv2.imread('1111.png',cv2.IMREAD_GRAYSCALE)
#jj = cv2.imread('sudoku.jpg',cv2.IMREAD_GRAYSCALE)
cv2.imshow('Problem',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#Adaptive Thresholding - calculates threshold for each pixel based on mean value of surrounding pixels
#Binary Thresholding - makes split of either 0/1 on basis of threshold measured from entire image

process = cv2.GaussianBlur(img.copy(),(9,9),0)
process = cv2.adaptiveThreshold(process,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
process = cv2.bitwise_not(process,process)

cv2.imshow('Filter',process)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:

image , external_contours, hier = cv2.findContours(process.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image ,contours, hier =  cv2.findContours(process.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
process = cv2.cvtColor(process,cv2.COLOR_GRAY2RGB)
cv2.imshow('Conters', process)
cv2.waitKey(0)
cv2.destroyAllWindows()

all_contours = cv2.drawContours(process.copy(), contours, -1, (255, 0, 0,), 2)
ext_contours = cv2.drawContours(process.copy(), external_contours, -1, (255, 0 ,0 ), 2)
cv2.imshow('All_Conters', all_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow('Ext_Conters', ext_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [5]:
#Sort by area, descending

contours = sorted(contours, key=cv2.contourArea, reverse=True) 
figure= contours[0]

# figure
maxi=-1
var=0
var1,var2=0,0
count=-1
for i in figure :
    count = count+1
    if (i[0][0]+i[0][1]>maxi) :
        maxi=i[0][0]+i[0][1]
        var=count
        var1=i[0][0]
        var2=i[0][1]

# print(maxi)
#print(var,var2)

bottom_right = var

maxi=-1
var=0
var1,var2=0,0
count=-1
for i in figure :
    count = count+1
    if (i[0][0]-i[0][1]>maxi) :
        maxi=i[0][0]-i[0][1]
        var=count
        var1=i[0][0]
        var2=i[0][1]
        
bottom_left = var
         
maxi=1000000
var=0
var1,var2=0,0
count=-1
for i in figure :
    count = count+1
    if (i[0][0]-i[0][1]<maxi) :
        maxi=i[0][0]-i[0][1]
        var=count
        var1=i[0][0]
        var2=i[0][1]
        
top_right = var    

maxi=1000000
var=0
var1,var2=0,0
count=-1
for i in figure :
    count = count+1
    if (i[0][0]+i[0][1]<maxi) :
        maxi=i[0][0]+i[0][1]
        var=count
        var1=i[0][0]
        var2=i[0][1]
        
top_left = var 

print("\n"+str(bottom_right)+" "+str(bottom_left)+" "+str(top_right)+" "+str(top_left))

radius=5
colour=(0, 0, 255)

points = [figure[top_left][0], figure[top_right][0], figure[bottom_right][0], figure[bottom_left][0]]

if len(img.shape) == 2:
	img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
elif img.shape[2] == 1:
	img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
for point in points:
		img = cv2.circle(img, tuple(int(x) for x in point), radius, colour, -1)  
        
cv2.imshow('image', img) 
cv2.waitKey(0) 
cv2.destroyAllWindows()       


3 4 2 0


In [6]:


Max_side = max ( [
    np.sqrt((points[0][0]-points[3][0])**2 + (points[0][1]-points[3][1])**2),
    np.sqrt((points[1][0]-points[2][0])**2 + (points[1][1]-points[2][1])**2),
    np.sqrt((points[2][0]-points[3][0])**2 + (points[2][1]-points[3][1])**2),
    np.sqrt((points[0][0]-points[1][0])**2 + (points[0][1]-points[1][1])**2),
] )

source_rect = np.array(np.array([points[0], points[3], points[2], points[1]], dtype='float32'))
destination_square = np.array([[0, 0], [Max_side - 1, 0], [Max_side - 1, Max_side - 1], [0, Max_side - 1]], dtype='float32')

#Comparing points
m = cv2.getPerspectiveTransform(source_rect , destination_square)

NEW_IMG = cv2.warpPerspective(img, m, (int(Max_side), int(Max_side)))
mmm = cv2.warpPerspective(process, m, (int(Max_side), int(Max_side)))

cv2.imshow('image', mmm) 
cv2.waitKey(0) 
cv2.destroyAllWindows()  


In [7]:

small_square = list()
img = NEW_IMG
side = img.shape

side

side= side[0]/9

for i in range(9) :
    for j in range(9) :
        point1=(i*side,j*side)
        point2=((i+1)*side,(j+1)*side)
        small_square.append((point1,point2))
        
rects = small_square    

colour=255

for rect in rects:
		img = cv2.rectangle(img, tuple(int(x) for x in rect[0]), tuple(int(x) for x in rect[1]), colour)

cv2.imshow('image', img) 
cv2.waitKey(0) 
cv2.destroyAllWindows()  

jj=img

In [8]:
height, width = img.shape[:2]
print(img.shape)
# Let's get the starting pixel coordiantes (top left of cropped top)
start_row, start_col = int(0), int(0)
# Let's get the ending pixel coordinates (bottom right of cropped top)
end_row, end_col = int(height / 9), int(width)
cropped_top = img[start_row:end_row , start_col:end_col]
print(start_row, end_row) 
print(start_col, end_col)

cv2.imshow("Cropped Top", cropped_top) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

# Let's get the starting pixel coordiantes (top left of cropped b0ottom)
start_row, start_col = int(height * .5), int(0)
# Let's get the ending pixel coordinates (bottom right of cropped bottom)
end_row, end_col = int(height), int(width)
cropped_bot = img[start_row:end_row , start_col:end_col]
print (start_row, end_row )
print (start_col, end_col)

cv2.imshow("Cropped Bot", cropped_bot) 
cv2.waitKey(0) 
cv2.destroyAllWindows()

(479, 479, 3)
0 53
0 479
239 479
0 479


In [9]:
height, width = img.shape[:2]
z=height/9


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def image_ocr(image):
    """
    Arguments:
    image: takes the croped cell from the folder.
    Returns
    return_string: Here the OCR recognise and converts the image of digits into string 
    and return it as a set of string 
    """
    #image = cv2.imread(image)
    return_string = pytesseract.image_to_string(image, lang='eng', config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
    return return_string

board=[]
for i in range (0,9):
        board.append([])
        for j in range(0,9):
            board[i].append(0)

            
print('Please wait, while the digit is completely recognised -- \n')            
for i in range(9) :
    for j in range(9) :
        row1=int((i)*z)
        row2=int((i+1)*z)
        coloumn1=int((j)*z)
        coloumn2=int((j+1)*z)
        cropped_bot = img[row1:row2 , coloumn1:coloumn2]
        imm = img[row1:row2 , coloumn1:coloumn2]
        process = cv2.GaussianBlur(imm.copy(),(9,9),0)
       # process = cv2.adaptiveThreshold(process,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        process = cv2.bitwise_not(process,process)
#         cv2.imshow("Cropped Bot", imm) 
#         cv2.waitKey(0) 
#         cv2.destroyAllWindows()
        text = image_ocr(imm)
        if text!="" :
              board[i][j]=int(text)
        else :
              board[i][j]=int(0)
        print((text),end=" ")
    print()   

    
print('Thank You for Your Patience')
#         cv2.imshow("Cropped Bot", cropped_bot) 
#         cv2.waitKey(0) 
#         cv2.destroyAllWindows()

Please wait, while the digit is completely recognised -- 

0 0 0 8 0 1 0  0 
0 0 0 0  0 0 4 3 
5 0 0 0  0 0 0 0 
0 0 0 0 7 0 8 0 0 
 0 0 0 0  1  0 
0 2 0 0 3 0 0 0 0 
6 0 0 0  0 0 0 8 
0  3 4 0 0 0 0 0 
 0  2   6  0 
Thank You for Your Patience


In [10]:
cropped_bot = img
cv2.imshow("Cropped Bot", cropped_bot) 
cv2.waitKey(0) 
cv2.destroyAllWindows()
imm = cropped_bot
text = image_ocr(imm)
print(text)

In [11]:
print('Board Looks Like this -- ')
print(board)
board1 = board

Board Looks Like this -- 
[[0, 0, 0, 8, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 4, 3], [5, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 7, 0, 8, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 2, 0, 0, 3, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 3, 4, 0, 0, 0, 0, 0], [0, 0, 0, 2, 0, 0, 6, 0, 0]]


In [12]:
from copy import copy, deepcopy

SUDOKU_BOARD= deepcopy(board)
ROW1=0
COLOUMN1=0

List_POSSIBLE_VALUES = [1,2,3,4,5,6,7,8,9]

def BACKTRACKING_IS_SAFE(ROW1,COLOUMN1,NUMBER) :
	for N in range(9) :
		if SUDOKU_BOARD[ROW1][N] == NUMBER or SUDOKU_BOARD[N][COLOUMN1] == NUMBER :
			return False

	Small_Grid_S = ROW1 - (ROW1 % 3) 
	Small_Grid_E = COLOUMN1 - (COLOUMN1 % 3)

	for I in range(3) :
		for J in range(3) :
			if SUDOKU_BOARD[I+Small_Grid_S][J+Small_Grid_E] == NUMBER :
				return False

	return True

def SUDOKU_SOLVE() :

	ROW = 0
	COLOUMN = 0
	flag = 0
	for ROW in range(9) : 
		flag = 0
		for COLOUMN in range(9) :
			if SUDOKU_BOARD[ROW][COLOUMN] == 0 :
				ROW1=ROW
				COLOUMN1=COLOUMN
				flag = 1
				break
		if flag == 1 :
			break							
	


	if flag == 0 :
		return True	

	for NUMBE in List_POSSIBLE_VALUES : 

		if BACKTRACKING_IS_SAFE(ROW1,COLOUMN1,NUMBE) :	
			
			SUDOKU_BOARD[ROW1][COLOUMN1]=NUMBE

			if SUDOKU_SOLVE() :
				return True	

			SUDOKU_BOARD[ROW1][COLOUMN1] = 0


	return False



if __name__ == "__main__" :

	if SUDOKU_SOLVE() == True :

		for ROW in range(9) :
			if ROW == 0 or ROW == 3 or ROW == 6 : 
				 for i in range(12) :  print(" _",end="")
			print()	  
			print ("|",end=" ")
			for COLOUMN in range(9) :
				if COLOUMN == 3 or COLOUMN == 6 : print("|",end=" ")
				print(SUDOKU_BOARD[ROW][COLOUMN],end=" ")
				if COLOUMN == 8 : print("|",end=" ")
			print()

		for i in range(12) :  print(" _",end="")	

	else :

		print('There Is No Possible Solution Possible For This Sudoku Puzzle')

        

 _ _ _ _ _ _ _ _ _ _ _ _
| 2 3 4 | 8 5 1 | 7 6 9 | 

| 1 6 8 | 7 2 9 | 5 4 3 | 

| 5 7 9 | 3 4 6 | 2 8 1 | 
 _ _ _ _ _ _ _ _ _ _ _ _
| 3 5 1 | 6 7 4 | 8 9 2 | 

| 4 8 6 | 5 9 2 | 1 3 7 | 

| 9 2 7 | 1 3 8 | 4 5 6 | 
 _ _ _ _ _ _ _ _ _ _ _ _
| 6 4 2 | 9 1 5 | 3 7 8 | 

| 8 1 3 | 4 6 7 | 9 2 5 | 

| 7 9 5 | 2 8 3 | 6 1 4 | 
 _ _ _ _ _ _ _ _ _ _ _ _

In [14]:
import pygame

board=board1
pygame.init()

Display = pygame.display.set_mode((800,800))

pygame.display.set_caption("Chess-Board")

Display.fill((202,202,202))
pygame.display.update()

boardLength = 8
size=80
cnt=0

one = pygame.image.load('one.png')
two = pygame.image.load('two.png')
three = pygame.image.load('three.png')
four = pygame.image.load('four.png')
five = pygame.image.load('five.png')
six = pygame.image.load('six.png')
seven = pygame.image.load('seven.png')
eight = pygame.image.load('eight.png')
nine = pygame.image.load('nine.png')

for i in range(1,boardLength+1):
    for z in range(1,boardLength+1):
            pygame.draw.rect(Display, (255,255,255),[size*z,size*i,size,size])
            if SUDOKU_BOARD[i-1][z-1] == 1 :  Display.blit(one, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 2 :  Display.blit(two, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 3 :  Display.blit(three, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 4 :  Display.blit(four, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 5 :  Display.blit(five, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 6 :  Display.blit(six, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 7 :  Display.blit(seven, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 8 :  Display.blit(eight, ((size*z+5,size*i+5)))
            if SUDOKU_BOARD[i-1][z-1] == 9 :  Display.blit(nine, ((size*z+5,size*i+5)))


one1 = pygame.image.load('one1.png')
two1 = pygame.image.load('two1.png')
three1 = pygame.image.load('three1.png')
four1 = pygame.image.load('four1.png')
five1 = pygame.image.load('five1.png')
six1 = pygame.image.load('six1.png')
seven1 = pygame.image.load('seven1.png')
eight1 = pygame.image.load('eight1.png')
nine1 = pygame.image.load('nine1.png')
                
                
for i in range(1,boardLength+1):
    for z in range(1,boardLength+1):
            pygame.draw.rect(Display, (255,255,255),[size*z,size*i,size,size])
            if board[i-1][z-1] != 0 :
                if board[i-1][z-1] == 1 :  Display.blit(one1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 2 :  Display.blit(two1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 3 :  Display.blit(three1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 4 :  Display.blit(four1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 5 :  Display.blit(five1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 6 :  Display.blit(six1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 7 :  Display.blit(seven1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 8 :  Display.blit(eight1, ((size*z+5,size*i+5)))
                if board[i-1][z-1] == 9 :  Display.blit(nine1, ((size*z+5,size*i+5)))
            else :
                if SUDOKU_BOARD[i-1][z-1] == 1 :  Display.blit(one, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 2 :  Display.blit(two, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 3 :  Display.blit(three, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 4 :  Display.blit(four, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 5 :  Display.blit(five, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 6 :  Display.blit(six, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 7 :  Display.blit(seven, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 8 :  Display.blit(eight, ((size*z+5,size*i+5)))
                if SUDOKU_BOARD[i-1][z-1] == 9 :  Display.blit(nine, ((size*z+5,size*i+5)))


pygame.draw.rect(Display,(50,100,200),[size,size,boardLength*size,boardLength*size],1)

pygame.display.update()

gameExit = False

while not gameExit:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            gameExit = True

pygame.quit()
quit()



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
